### Explore Batter Data

In [100]:
import pandas as pd
import numpy as np
from copy import deepcopy

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',None)

In [101]:
batter_profiles = pd.read_csv('milb_batter_summaries.csv')
batter_seasons = pd.read_csv('milb_all_batter_seasons.csv')

In [102]:
len(batter_profiles.milb_id.unique())

17525

__Helper Functions__

Add Useful Data Fields

In [103]:
def add_season_number(df):
    first_year = pd.DataFrame()
    first_year['FirstYear'] = df.groupby(['milb_id']).min()['Year']
    first_year = first_year.reset_index()
    df = df.merge(first_year, how = 'outer')
    
    df['SeasonNumber'] = df['Year'] - df['FirstYear'] + 1
    #df = df.drop('First_Year',axis = 1)
    return df

def add_success_year(df):
    success_year = pd.DataFrame()
    success_year['FirstYear'] = df.groupby(['milb_id']).min()['Year']
    first_year = first_year.reset_index()
    df = df.merge(first_year, how = 'outer')
    
    df['SeasonNumber'] = df['Year'] - df['FirstYear'] + 1
    #df = df.drop('First_Year',axis = 1)
    return df


Add Stat Calulation Functions

In [104]:
## Batting Average
# Number of Hits (divided by) Number of At Bats

# Calculate the number of single from other hit data
def calc_singles(df):
    df['1B'] = df['H'] - df['2B'] - df['3B'] - df['HR']
    return df

def calc_BA(df):
    ba_series = df['H']/df['AB']
    return ba_series
    
## Slugging Percentage
# Number of (Singles + [2 x Doubles] +[ 3 x Triples] + [4 x Home Runs]) divided by At Bats  
def calc_SLG(df):
    slg_series = (df['1B'] + 2*df['2B'] + 3*df['3B'] + 4*df['HR']) / df['AB']
    return slg_series

## On Base Percentage
# (Hits + Walks + Hit-By-Pitch) divided by (At Bats + Walks+ Hit-By-Pitch + Sac Flys)
def calc_OBP(df):
    obp_series = (df['H'] + df['BB'] + df['HBP']) \
        / (df['AB'] + df['BB'] + df['HBP']+df['SF']) 
    return obp_series

## On Base Plus Slugging 
def calc_OPS(df):
     # On Base Plus Slugging 
    ops_series = calc_OBP(df) + calc_SLG(df)
    return ops_series

Add Aggregation Function

In [123]:
# Aggregate year by year stats into full career data.
# Requires the recalculation of rate based stats from their definitions.
def make_season_stats(df):
    
    counting_stats = ['G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB',
       'CS', 'BB', 'SO','TB', 'GDP', 'HBP','SH', 'SF', 'IBB']
    
    
    if 'LevNum' in df.columns:
        df['LevWeight'] = df['LevNum']*df['PA']
    
    
    # Make groupby object to only do aggregation once
    grouped_obj = df.groupby(['milb_id','Year'])
    
    df_grouped = grouped_obj.sum()
    
    df_grouped['1B'] = calc_singles(df_grouped)
    
    # Batting Average 
    df_grouped['BA'] = calc_BA(df_grouped)
    
    # On Base Percentage
    df_grouped['OBP'] = calc_OBP(df_grouped)
    
    # Slugging Percentage
    df_grouped['SLG'] = calc_SLG(df_grouped)
    
    # On Base Plus Slugging 
    df_grouped['OPS'] = calc_OPS(df_grouped)
    
    # Other BIO 
    # Fields where max is most important 
    test_fields = ['Age','FirstYear',
                   'SeasonNumber','SuccessYear']
    for tf in test_fields:
        if tf in df_grouped.columns:
            df_grouped[tf] = grouped_obj[tf].max()
            
    # fields where min is most importand
    test_fields2 = ['AgeDif']
    for tf in test_fields2:
        if tf in df_grouped.columns:
            df_grouped[tf] = grouped_obj[tf].min()
                  
    # Minor League Level Attributs
    if 'LevNum' in df.columns:
        df_grouped['LevWeighted'] = df_grouped['LevWeight']/df_grouped['PA']
        
        df_grouped['LevCount'] = grouped_obj.count()['Lev']
        df_grouped.drop('LevWeight', axis=1, inplace=True)
    
    df_grouped.reset_index(inplace=True)
    
    return df_grouped


__Filtering and Aggregation__

In [106]:
batter_seasons_filt = deepcopy(batter_seasons)

# Drop Aggregated Seasons. Will Recompute
batter_seasons_filt = batter_seasons_filt[
    batter_seasons_filt.Tm.str.contains('Teams') == False]

In [107]:
# Map Each League to a level
lev_dict = {'FRk':0,'Rk':1,'A-':2,'A':3,'A+':4,'AA':5,'AAA':6}
level_map = pd.Series(batter_seasons_filt.loc[:,('Lev')].map(lev_dict))
batter_seasons_filt['LevNum'] = level_map

# Drop Leagues that are "Atypical"
weird_leagues = ['Ind','Rk','FRk','WRk','FgW','Fal','Fgn','Wtr']
batter_seasons_filt = batter_seasons_filt[
    batter_seasons_filt['Lev'].isin(weird_leagues) == False]

In [108]:
# Remove Players Who had their first season before 1990.
batter_seasons_filt = add_season_number(batter_seasons_filt)
batter_seasons_filt = batter_seasons_filt[
    batter_seasons_filt['FirstYear'] > 1990]

In [109]:
# Remove Seasons that occured after player made the majors
majors_df = batter_seasons_filt[
    batter_seasons_filt.Lev == 'MLB'][['milb_id','Year']]

mlb_debuts = majors_df.groupby('milb_id').min().reset_index()
mlb_debuts.rename(columns={'Year':'SuccessYear'},inplace=True)

batter_seasons_filt = batter_seasons_filt.merge(
    mlb_debuts, how = 'left',on= 'milb_id')

batter_seasons_filt = batter_seasons_filt[
    (batter_seasons_filt['SuccessYear'] < batter_seasons_filt['Year']) | 
    pd.isnull(batter_seasons_filt['SuccessYear'])] 


In [124]:
batter_seasons_total = make_season_stats(batter_seasons_filt)

In [125]:
batter_seasons_total

,milb_id,Year,Age,AgeDif,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,TB,GDP,HBP,SH,SF,IBB,LevNum,FirstYear,SeasonNumber,SuccessYear,1B,LevWeighted,LevCount
0,aalber001bra,1998,21,-0.4,22.0,54.0,46.0,3.0,8.0,2.0,0.0,0.0,1.0,1.0,2.0,4.0,12.0,0.173913,0.269231,0.543478,0.812709,10.0,2.0,2.0,2.0,0.0,0.0,3.0,1998,1,NaN,21.0,3.000000,1
1,aaron001ogi,2000,20,-2.7,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,20.000000,20.000000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2000,1,NaN,20.0,4.000000,1
2,abad001fau,2002,29,1.9,111.0,415.0,352.0,50.0,106.0,28.0,2.0,11.0,70.0,0.0,3.0,57.0,44.0,0.301136,0.402410,0.383523,0.785932,171.0,7.0,4.0,0.0,2.0,1.0,6.0,1994,9,2001.0,29.0,6.000000,1
3,abad001fau,2003,30,1.3,143.0,592.0,521.0,79.0,155.0,35.0,3.0,13.0,93.0,0.0,4.0,57.0,72.0,0.297505,0.366723,0.366603,0.733326,235.0,16.0,4.0,3.0,7.0,8.0,6.0,1994,10,2001.0,60.0,5.807432,2
4,abad001fau,2004,31,3.8,99.0,349.0,301.0,45.0,88.0,15.0,1.0,12.0,49.0,4.0,1.0,40.0,52.0,0.292359,0.382184,0.372093,0.754277,141.0,9.0,5.0,1.0,2.0,1.0,6.0,1994,11,2001.0,31.0,6.000000,1
5,abad001fau,2005,32,4.7,121.0,474.0,423.0,68.0,124.0,29.0,1.0,20.0,85.0,3.0,4.0,44.0,60.0,0.293144,0.360759,0.408983,0.769743,215.0,16.0,3.0,0.0,4.0,5.0,6.0,1994,12,2001.0,32.0,6.000000,1
6,abad001fau,2006,33,3.8,88.0,309.0,269.0,36.0,71.0,12.0,0.0,9.0,32.0,0.0,1.0,28.0,31.0,0.263941,0.345395,0.468401,0.813796,110.0,6.0,6.0,5.0,1.0,1.0,6.0,1994,13,2001.0,66.0,5.902913,2
7,abad001fau,2007,34,7.0,83.0,300.0,269.0,49.0,85.0,12.0,0.0,12.0,59.0,6.0,2.0,24.0,35.0,0.315985,0.367893,0.394052,0.761945,133.0,6.0,1.0,1.0,5.0,6.0,6.0,1994,14,2001.0,34.0,6.000000,1
8,abad001fau,2008,35,5.5,30.0,121.0,107.0,10.0,21.0,5.0,0.0,0.0,14.0,0.0,1.0,10.0,19.0,0.196262,0.283333,0.420561,0.703894,26.0,3.0,3.0,1.0,0.0,1.0,6.0,1994,15,2001.0,35.0,6.000000,1
9,abate001mic,1999,20,-1.3,57.0,206.0,185.0,34.0,47.0,15.0,0.0,5.0,23.0,1.0,2.0,17.0,57.0,0.254054,0.323529,0.378378,0.701908,77.0,2.0,2.0,2.0,0.0,1.0,2.0,1999,1,NaN,20.0,2.000000,1


In [ ]:
batter_seasons_total

In [ ]:
# 'CREATE TABLE IF NOT EXISTS BatterSeasons (
#     Year int DEFAULT NULL,
#     Age int DEFAULT NULL,
#     AgeDif float DEFAULT NULL,
#     G int DEFAULT NULL,
#     PA int DEFAULT NULL,
#     AB int DEFAULT NULL,
#     R int DEFAULT NULL,
#     H int DEFAULT NULL,
#     2B int DEFAULT NULL,
#     3B int DEFAULT NULL,
#     HR int DEFAULT NULL,
#     RBI int DEFAULT NULL,
#     SB int DEFAULT NULL,
#     CS int DEFAULT NULL,
#     BB int DEFAULT NULL,
#     SO int DEFAULT NULL,
#     BA float DEFAULT NULL,
#     OBP float DEFAULT NULL,
#     SLG float DEFAULT NULL,
#     OPS float DEFAULT NULL,
#     TB int DEFAULT NULL,
#     GDP int DEFAULT NULL,
#     HBP int DEFAULT NULL,
#     SH int DEFAULT NULL,
#     SF int DEFAULT NULL,
#     IBB int DEFAULT NULL,
#     Tm varchar(20) NOT NULL,
#     Lg varchar(20) NOT NULL,
#     Lev varchar(20) NOT NULL,
#     Aff varchar(20) NOT NULL,
#     mlb_id varchar(20) NOT NULL,
#     milb_id varchar(20) NOT NULL],
    
    
    
    
    
    
#     playerID varchar(20) NOT NULL,
#     yearID int NOT NULL,
#     gameNum varchar(20) NOT NULL,
#     gameID varchar(12) DEFAULT NULL,
#     teamID text DEFAULT NULL,
#     lgID text DEFAULT NULL,
#     GP varchar(20) DEFAULT NULL,
#     startingPos varchar(20) DEFAULT NULL,
#     PRIMARY KEY (playerID,yearID,gameNum)
# );'